# Mastery Checkpoint 2
# Random Forest 

CSC 466: KDD
Asmita Sharma

In [2]:
from pathlib import Path
home = str(Path.home())
import warnings
warnings.simplefilter('ignore')

In [3]:
%load_ext autoreload
%autoreload 2
import Lab5_helper
import random_forest_helper

In [4]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from sklearn.model_selection import train_test_split

titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#preprocessing
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
titanic_df2 = titanic_df.loc[:,features]
titanic_df2['CabinLetter'] = titanic_df2['Cabin'].str.slice(0,1)
X = titanic_df2.drop('Cabin',axis=1)
X['CabinLetter'] = X['CabinLetter'].fillna("?")
X['Pclass'] = X['Pclass'].astype(str)
X['SibSp'] = X['SibSp'].astype(str)
X['Parch'] = X['Parch'].astype(str)
X = X.dropna()
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinLetter
0,3,male,22.0,1,0,7.2500,S,?
1,1,female,38.0,1,0,71.2833,C,C
2,3,female,26.0,0,0,7.9250,S,?
3,1,female,35.0,1,0,53.1000,S,C
4,3,male,35.0,0,0,8.0500,S,?


In [6]:
t = titanic_df.loc[X.index,'Survived']
X2 = pd.get_dummies(X)

In [7]:
X_train, X_test, t_train, t_test = train_test_split(X2, t, test_size=0.3)


# Decision Tree Regressor

In [8]:
trees = random_forest_helper.make_trees(X_train, t_train, 100)

In [9]:
default = t_train.mode().get(0)
predictions = random_forest_helper.reg_predict_all(trees, X_test, default)
final_predictions = random_forest_helper.reg_prediction(predictions)

In [10]:
accuracy_dt = sum(final_predictions == t_test) / len(t_test)
accuracy_dt * 100, '%'

(65.42056074766354, '%')

In [11]:
import Lab5_helper

# Boosting 

In [12]:
default = t_train.mode().get(0)
X_train2, X_val, t_train2, t_val = train_test_split(X_train, t_train, test_size=0.3)
trees,train_RMSEs,val_RMSEs = Lab5_helper.make_trees_boost(X_train2, X_val, t_train2, t_val, max_ntrees=100)
trees = Lab5_helper.cut_trees(trees,val_RMSEs)


In [13]:
pred = Lab5_helper.make_prediction_boost(trees,X_test)
pred = np.around(pred)


In [14]:
accuracy_boosting = sum(pred == t_test) / len(t_test)
accuracy_boosting * 100, '%'

(78.97196261682244, '%')

# Bagging

In [15]:
trees = Lab5_helper.make_trees(X_train, t_train)

In [16]:
pred = Lab5_helper.make_prediction(trees, X_test)
pred = np.around(pred)

In [18]:
accuracy_bagging = sum(t_test == pred) / len(pred)
accuracy_bagging * 100, '%'

(75.23364485981308, '%')

# Netflix Dataset

In [19]:
# Top 10 Shows on Netflix USA Dataset

In [20]:
netflix_top10_df = pd.read_csv('netflix daily top 10.csv')
netflix_top10_df

,As of,Rank,Year to Date Rank,Last Week Rank,Title,Type,Netflix Exclusive,Netflix Release Date,Days In Top 10,Viewership Score
0,2020-04-01,1,1,1,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",9,90
1,2020-04-01,2,2,-,Ozark,TV Show,Yes,"Jul 21, 2017",5,45
2,2020-04-01,3,3,2,All American,TV Show,NaN,"Mar 28, 2019",9,76
3,2020-04-01,4,4,-,Blood Father,Movie,NaN,"Mar 26, 2020",5,30
4,2020-04-01,5,5,4,The Platform,Movie,Yes,"Mar 20, 2020",9,55
...,...,...,...,...,...,...,...,...,...,...
7095,2022-03-11,6,5,1,Worst Roommate Ever,TV Show,Yes,"Mar 1, 2022",10,81
7096,2022-03-11,7,7,2,Vikings: Valhalla,TV Show,Yes,"Feb 25, 2022",14,100
7097,2022-03-11,8,8,-,Shooter,Movie,NaN,"Aug 1, 2014",3,7
7098,2022-03-11,9,9,7,Shrek 2,Movie,NaN,"Mar 1, 2022",10,33


In [21]:
# preprocessing
netflix_top10_df = netflix_top10_df.replace('-', np.nan)
netflix_top10_df = netflix_top10_df.dropna()
netflix_top10_df

#YTDR, type, exclusive, rank, predict viewership score

,As of,Rank,Year to Date Rank,Last Week Rank,Title,Type,Netflix Exclusive,Netflix Release Date,Days In Top 10,Viewership Score
0,2020-04-01,1,1,1,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",9,90
4,2020-04-01,5,5,4,The Platform,Movie,Yes,"Mar 20, 2020",9,55
7,2020-04-01,8,7,5,Love is Blind,TV Show,Yes,"Feb 13, 2020",9,40
10,2020-04-02,1,1,1,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",10,100
18,2020-04-02,9,5,4,The Platform,Movie,Yes,"Mar 20, 2020",10,57
...,...,...,...,...,...,...,...,...,...,...
7086,2022-03-10,7,6,2,Vikings: Valhalla,TV Show,Yes,"Feb 25, 2022",13,96
7093,2022-03-11,4,4,3,Inventing Anna,TV Show,Yes,"Feb 11, 2022",28,250
7094,2022-03-11,5,6,6,Love is Blind,TV Show,Yes,"Feb 13, 2020",46,312
7095,2022-03-11,6,5,1,Worst Roommate Ever,TV Show,Yes,"Mar 1, 2022",10,81


In [22]:
# Preprocessing
features = ["Rank", "Year to Date Rank", "Last Week Rank", "Viewership Score"]
netflix = netflix_top10_df.loc[:, features]

X = netflix.loc[:, ["Rank", "Year to Date Rank",  "Last Week Rank"]]
y = netflix["Viewership Score"]

X_train, X_test, t_train, t_test = train_test_split(X, y, test_size=0.3)

In [23]:
X_train

,Rank,Year to Date Rank,Last Week Rank
6235,6,5,2
404,5,4,3
493,4,4,1
3471,2,1,1
955,6,5,2
...,...,...,...
6650,1,1,9
6526,7,7,6
1559,10,9,3
5642,3,3,2


# Decision Tree 

In [24]:
trees = random_forest_helper.make_trees(X_train, t_train, ntrees=100)


In [25]:
default = t_train.mode().get(0)
predictions = random_forest_helper.reg_predict_all(trees, X_test, default)
final_predictions = random_forest_helper.reg_prediction(predictions)

In [26]:
accuracy_dt = sum(final_predictions == t_test) / len(t_test)
accuracy_dt * 100, '%'

(1.530612244897959, '%')

# Boosting

In [39]:
default = t_train.mode().get(0)
X_train2, X_val, t_train2, t_val = train_test_split(X_train, t_train, test_size=0.3)
trees,train_RMSEs,val_RMSEs = Lab5_helper.make_trees_boost(X_train2, X_val, t_train2, t_val, max_ntrees=100)
trees = Lab5_helper.cut_trees(trees,val_RMSEs)


In [40]:
pred = Lab5_helper.make_prediction_boost(trees,X_test)
pred = np.around(pred)


In [41]:
accuracy_boosting = sum(pred == t_test) / len(t_test)
accuracy_boosting * 100, '%'

(0.6802721088435374, '%')

# Bagging

In [30]:
trees = Lab5_helper.make_trees(X_train, t_train)

In [31]:
pred = Lab5_helper.make_prediction(trees, X_test)
pred = np.around(pred)

In [32]:
accuracy_bagging = sum(t_test == pred) / len(pred)
accuracy_bagging * 100, '%'

(0.3401360544217687, '%')